In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from pathlib import Path
import cv2
from functools import partial

from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
from watch_recognition.augmentations import set_shapes

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100
NUM_KEYPOINTS = 4 * 2  # 24 pairs each having x and y coordinates

In [ ]:
def get_model():
    backbone = tf.keras.applications.EfficientNetB0(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
    )
    

    for layer in backbone.layers:
        if "project_conv" in layer.name:
            print(layer.name, layer.output.shape)
    outputs = [
        backbone.get_layer(layer_name).output
        for layer_name in ["block7a_project_conv"]
    ]
    base_model = tf.keras.Model(inputs=[backbone.inputs], outputs=outputs)
    base_model.trainable=False
    inputs = tf.keras.Input(
        shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    x  = base_model(inputs)
    options = {
        'kernel_initializer': tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None),
        'bias_initializer':'zeros',
    }
    
    for i in range(3):
        x = tf.keras.layers.Conv2D(filters=160, kernel_size=3, activation=None)(x)
#         x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

#     x = tf.keras.layers.Conv2D(
#         filters=NUM_KEYPOINTS, kernel_size=5, strides=1, activation=None
#     )(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

    x = tf.keras.layers.Conv2D(
        filters=NUM_KEYPOINTS,
        kernel_size=1,
        activation='sigmoid'
    )(x)
    output = tf.keras.layers.Flatten()(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=output)

    return keras.Model(inputs, output, name="keypoint_detector")

In [ ]:
get_model().summary()

In [ ]:
X, y, _ = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/train"), autorotate=True)
y = y[:, :, :2].reshape(-1, NUM_KEYPOINTS) / IMG_SIZE
X.shape, y.shape

In [ ]:
X_val, y_val, _ = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/validation"), autorotate=True
)
y_val = y_val[:, :, :2].reshape(-1, NUM_KEYPOINTS) / IMG_SIZE
X_val.shape, y_val.shape

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X, y))


In [ ]:
img, kp = next(iter(train_ds))

In [ ]:
kp.numpy().reshape((-1,2))

In [ ]:
def vis_keypoints(image, keypoints, color=(0, 255, 0)):
    image = image.copy().astype("uint8")
    diameter = int(np.mean([image.shape[0], image.shape[1]]) / 50)
    keypoints = keypoints * IMG_SIZE
    for kp in keypoints:
        x, y = kp[0], kp[1]
        if 0 < x < image.shape[0] and 0 < y < image.shape[1]:
            x_int = int(x)
            y_int = int(y)
            cv2.circle(image, (x_int, y_int), diameter, color, -1)
        else:
            print(f"kp {x},{y} ignored")

    plt.figure(figsize=(8, 8))
    plt.axis("off")
    plt.imshow(image)


vis_keypoints(X[0], y[0].reshape((-1, 2)))

In [ ]:
vis_keypoints(X_val[0], y_val[0].reshape((-1, 2)))

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_alb = train_ds.shuffle(8 * BATCH_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_alb

In [ ]:
y_val.shape

In [ ]:
from datetime import datetime

In [ ]:
model = get_model()
start = datetime.now()
TYPE = 'keypoint_regression'
MODEL_NAME = 'efficientnetb0'
logdir = f"tensorboard_logs/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}"
print(logdir)

model_path = f'models/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}.h5'

model.compile(
    loss=tf.keras.losses.mean_squared_error,
    optimizer=keras.optimizers.Adam(),
    metrics=[
        "mae",
        'mse',
        "mean_absolute_percentage_error",
    ],
)
model.fit(
    X,
    y,
    validation_data=(X_val, y_val),
    epochs=EPOCHS//2,
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=model_path,
            save_weights_only=False,
            monitor='val_loss',
            save_best_only=True,
        ),
    ]
)

In [ ]:
sample_val_images, sample_val_keypoints = X[:1], y[:1]
sample_val_keypoints = sample_val_keypoints[0].reshape(-1, 2)
predictions = model.predict(sample_val_images).reshape(-1, 2)

In [ ]:
X[:1].shape

In [ ]:
# Ground-truth
vis_keypoints(sample_val_images[0], sample_val_keypoints)

In [ ]:
# Predictions
vis_keypoints(sample_val_images[0], predictions)

In [ ]:
predictions

In [ ]:
pred = model.predict(sample_val_images)

In [ ]:
pred

In [ ]:
np.mean((y[0] - pred)**2)

In [ ]:
mse = tf.keras.losses.MeanSquaredError()

In [ ]:
mse(y[0],pred).numpy()

In [ ]:
tf.keras.losses.mean_squared_error(y[0],pred).numpy()

In [ ]:
pred = model.predict(X_val)

In [ ]:
pred.shape

In [ ]:
losses = tf.keras.losses.mean_squared_error(y_val,pred).numpy()

In [ ]:
vis_keypoints(X_val[losses.argmax()], y_val[losses.argmax()].reshape((-1, 2)))

In [ ]:
vis_keypoints(X_val[losses.argmax()], pred[losses.argmax()].reshape((-1, 2)))